In [32]:
import pandas as pd

In [18]:
df_0 = pd.read_csv("pm fmkn upd.csv")

In [19]:
df_0

,Unnamed: 0,Титульная страница,Раздел 1.,1.1.,1.2.,1.3.,1.4.,Раздел 2.,2.1.,2.1.1,...,3.3.1,3.3.2,3.3.3,3.3.4,3.3.5,3.4.,3.4.1,3.4.2,3.4.3,Раздел 4.
0,0,санкт-петербургский государственный университе...,NaN,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,в результат обучение у обучаться формироваться...,"лекция (30 час.), практический занятие (14 час...",NaN,NaN,NaN,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",NaN,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",нет,"потоцкий ирина юрьевна, к.ф-м.н., доцент, i.po..."
1,1,санкт-петербургский государственный университе...,NaN,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,в результат обучение у обучаться формироваться...,"лекция (30 час.), практический занятие (14 час...",NaN,NaN,NaN,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",NaN,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",нет,"потоцкий ирина юрьевна, к.ф-м.н., доцент, i.po..."
2,2,санкт-петербургский государственный университе...,NaN,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,дисциплина способствовать формирование следующ...,"лекция (32 час.), практический занятие (16 час...",NaN,NaN,NaN,...,учебный аудитория для проведение учебный занят...,"стандартный оборудование, использовать для обу...",нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",NaN,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",• сайт научный библиотека им. м. горький спбгу...,"потоцкий ирина юрьевна, к.ф-м.н., доцент, шима..."
3,3,санкт-петербургский государственный университе...,NaN,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,дисциплина способствовать формирование следующ...,"лекция (32 час.), практический занятие (16 час...",NaN,NaN,NaN,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",NaN,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",нет,"потоцкий ирина юрьевна, к.ф-м.н., доцент, шима..."
4,4,санкт-петербургский государственный университе...,NaN,цель учебный дисциплина являться освоение студ...,"для понимание материал дисциплина необходимо, ...",в результат освоение дисциплина формироваться ...,"лекция (56 час.), практический занятие (56 час...",NaN,NaN,NaN,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",NaN,"1. бабаджанянц л.к., пупышева ю.а., пупышева ю...",1. арнольд в.и. математический метод классичес...,1. электронный учебник «классический механика»...,"бабаджанянц левон константинович, д.ф.-м.н., п..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4819,4819,санкт-петербургский государственный университе...,цель и задача учебный занятие учебный занятие ...,«готовый правильно использовать представление ...,NaN,по факт накопление 400 балл в соответствие с б...,NaN,учебный аудитория для проведение учебный занят...,балльный система контроль учебный занятие физи...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4820,4820,санкт-петербургский государственный университе...,цель и задача учебный занятие учебный занятие ...,«готовый правильно использовать представление ...,NaN,по факт накопление 400 балл в соответствие с б...,NaN,учебный аудитория для проведение учебный занят...,балльный система контроль у

In [20]:
df = df_0.drop(columns = ['Unnamed: 0'],axis = 1)

In [21]:
df

,Титульная страница,Раздел 1.,1.1.,1.2.,1.3.,1.4.,Раздел 2.,2.1.,2.1.1,2.2.,...,3.3.1,3.3.2,3.3.3,3.3.4,3.3.5,3.4.,3.4.1,3.4.2,3.4.3,Раздел 4.
0,санкт-петербургский государственный университе...,NaN,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,в результат обучение у обучаться формироваться...,"лекция (30 час.), практический занятие (14 час...",NaN,NaN,NaN,основной курс основный траектория очный форма ...,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",NaN,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",нет,"потоцкий ирина юрьевна, к.ф-м.н., доцент, i.po..."
1,санкт-петербургский государственный университе...,NaN,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,в результат обучение у обучаться формироваться...,"лекция (30 час.), практический занятие (14 час...",NaN,NaN,NaN,основной курс основный траектория очный форма ...,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",NaN,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",нет,"потоцкий ирина юрьевна, к.ф-м.н., доцент, i.po..."
2,санкт-петербургский государственный университе...,NaN,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,дисциплина способствовать формирование следующ...,"лекция (32 час.), практический занятие (16 час...",NaN,NaN,NaN,основной курс основный траектория очный форма ...,...,учебный аудитория для проведение учебный занят...,"стандартный оборудование, использовать для обу...",нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",NaN,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",• сайт научный библиотека им. м. горький спбгу...,"потоцкий ирина юрьевна, к.ф-м.н., доцент, шима..."
3,санкт-петербургский государственный университе...,NaN,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,дисциплина способствовать формирование следующ...,"лекция (32 час.), практический занятие (16 час...",NaN,NaN,NaN,основной курс основный траектория очный форма ...,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",NaN,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",нет,"потоцкий ирина юрьевна, к.ф-м.н., доцент, шима..."
4,санкт-петербургский государственный университе...,NaN,цель учебный дисциплина являться освоение студ...,"для понимание материал дисциплина необходимо, ...",в результат освоение дисциплина формироваться ...,"лекция (56 час.), практический занятие (56 час...",NaN,NaN,NaN,основной курс основный траектория очный форма ...,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",NaN,"1. бабаджанянц л.к., пупышева ю.а., пупышева ю...",1. арнольд в.и. математический метод классичес...,1. электронный учебник «классический механика»...,"бабаджанянц левон константинович, д.ф.-м.н., п..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4819,санкт-петербургский государственный университе...,цель и задача учебный занятие учебный занятие ...,«готовый правильно использовать представление ...,NaN,по факт накопление 400 балл в соответствие с б...,NaN,учебный аудитория для проведение учебный занят...,балльный система контроль учебный занятие физи...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4820,санкт-петербургский государственный университе...,цель и 

In [15]:
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import pymorphy2

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('russian')
stop = set(stopwords.words('russian'))

def preproc(text):
    for i in range(0, len(text)):
        if text[i] is not np.nan:
            morph = pymorphy2.MorphAnalyzer()
            ps = nltk.porter.PorterStemmer()
            text[i] = ' '.join([morph.parse(word)[0].normal_form for word in text[i].split()])
            tokens = tokenizer.tokenize(text[i])
            tokens = [token.strip() for token in tokens]
            cleaned_tokens = [token for token in tokens if token.lower() not in stopword_list]
            cleaned_text = ' '.join(cleaned_tokens)               
    return text


In [22]:
# df = df.apply(preproc)
df.to_csv("pm fmkn upd.csv", index=False)

In [23]:
df = df.fillna('')

In [24]:
df

,Титульная страница,Раздел 1.,1.1.,1.2.,1.3.,1.4.,Раздел 2.,2.1.,2.1.1,2.2.,...,3.3.1,3.3.2,3.3.3,3.3.4,3.3.5,3.4.,3.4.1,3.4.2,3.4.3,Раздел 4.
0,санкт-петербургский государственный университе...,,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,в результат обучение у обучаться формироваться...,"лекция (30 час.), практический занятие (14 час...",,,,основной курс основный траектория очный форма ...,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",нет,"потоцкий ирина юрьевна, к.ф-м.н., доцент, i.po..."
1,санкт-петербургский государственный университе...,,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,в результат обучение у обучаться формироваться...,"лекция (30 час.), практический занятие (14 час...",,,,основной курс основный траектория очный форма ...,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",нет,"потоцкий ирина юрьевна, к.ф-м.н., доцент, i.po..."
2,санкт-петербургский государственный университе...,,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,дисциплина способствовать формирование следующ...,"лекция (32 час.), практический занятие (16 час...",,,,основной курс основный траектория очный форма ...,...,учебный аудитория для проведение учебный занят...,"стандартный оборудование, использовать для обу...",нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",• сайт научный библиотека им. м. горький спбгу...,"потоцкий ирина юрьевна, к.ф-м.н., доцент, шима..."
3,санкт-петербургский государственный университе...,,цель учебный дисциплина являться обучение студ...,для успешный освоение курс учебный дисциплина ...,дисциплина способствовать формирование следующ...,"лекция (32 час.), практический занятие (16 час...",,,,основной курс основный траектория очный форма ...,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",,"1. новосёлов в.с., королёв в.с. аналитический ...","1. бабаджанянц л.к., потоцкий и.ю. управление ...",нет,"потоцкий ирина юрьевна, к.ф-м.н., доцент, шима..."
4,санкт-петербургский государственный университе...,,цель учебный дисциплина являться освоение студ...,"для понимание материал дисциплина необходимо, ...",в результат освоение дисциплина формироваться ...,"лекция (56 час.), практический занятие (56 час...",,,,основной курс основный траектория очный форма ...,...,"аудитория и помещения, предназначить для прове...",для проведение учебный занятие аудитория должн...,нет,нет,"мел и/ить фломастер цветные, губки, бумага фор...",,"1. бабаджанянц л.к., пупышева ю.а., пупышева ю...",1. арнольд в.и. математический метод классичес...,1. электронный учебник «классический механика»...,"бабаджанянц левон константинович, д.ф.-м.н., п..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4819,санкт-петербургский государственный университе...,цель и задача учебный занятие учебный занятие ...,«готовый правильно использовать представление ...,,по факт накопление 400 балл в соответствие с б...,,учебный аудитория для проведение учебный занят...,балльный система контроль учебный занятие физи...,,,...,,,,,,,,,,
4820,санкт-петербургский государственный университе...,цель и задача учебный занятие учебный занятие ...,«готовый правильно использовать представление ...,,по факт накопление 400 

In [25]:
df.to_csv("pm fmkn upd.csv", index=False)

In [26]:
df_1 = df.to_numpy().reshape(4824*31)

In [27]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

vector_size = 30
documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(df_1)]
model = Doc2Vec(documents, vector_size=vector_size, window=2, min_count=1, workers=4)
model.build_vocab(documents)

In [28]:
for epoch in range(10):
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=model.epochs)

In [29]:
model.save("pm_fmkn_trained upd")

# Проверка векторизации

In [1]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("pm_fmkn_trained upd")

In [28]:
import pandas as pd
mm = pd.read_csv('math-mech upd.csv')

In [29]:
mm.drop(index=0)

,Титульная страница,Раздел 1.,1.1.,1.2.,1.3.,1.4.,Раздел 2.,2.1.,2.1.1,2.2.,...,3.3.1,3.3.2,3.3.3,3.3.4,3.3.5,3.4.,3.4.1,3.4.2,3.4.3,Раздел 4.
1,санкт-петербургский государственный университе...,NaN,основный цель обучение иностранный обучаться р...,"обучающиеся, приступать к изучение курс русски...",по завершение курс обучение у учиться должный ...,основной форма обучение русский язык являться ...,NaN,предусмотреть учебный занятие с использование ...,NaN,каждый учебный год иметь семестровый и модульн...,...,стандартно оборудовать аудитория для практичес...,компьютерный класс с выход в internet.,не предусмотреть,не предусмотреть,не предусмотреть,NaN,1. барсуков-сергеев о.м. знакомый глаголы: пос...,"акишин т.е., алексеев н.н. пособие по обучение...",филологический портал: philology.ru справочно-...,"волков л.б., канд. филол. наук, доцент, доцент..."
2,санкт-петербургский государственный университе...,NaN,NaN,NaN,NaN,NaN,NaN,предусмотреть учебный занятие с использование ...,NaN,l-й блок «спортивный» l-й модуль «самосовершен...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,санкт-петербургский государственный университе...,NaN,цель и задача обучение английский язык формули...,уровень подготовленность обучаться определятьс...,в результат освоение программа обучаться должн...,"в связь с тем, что данный курс являться коммун...",NaN,предусмотреть учебный занятие с использование ...,NaN,NaN,...,в аудитория обязательно наличие доски. при про...,всё рабочий станция должный быть оснастить лиц...,специализировать лингафонный и мультимедийный ...,определяться в соответствие с состав закупка в...,расходный материал обеспечиваться в соответств...,NaN,определяться при составление профессионально-о...,общий курс (по выбор преподавателя): 1. amanda...,наиболее полный перечень информационный источн...,"кузнецов людмила борисовна, к.ф.н., доцент, до..."
4,санкт-петербургский государственный университе...,NaN,цель и задача обучение английский язык формули...,наличие компетенция в предел уровень в2.,в результат освоение программа обучаться должн...,"в связь с тем, что данный курс являться коммун...",NaN,NaN,NaN,для обучаться высокий уровень сформированность...,...,"в аудитория обязательно наличие доски, а также...",всё рабочий станция должный быть оснастить лиц...,специализировать лингафонный и мультимедийный ...,определяться в соответствие с требование текущ...,расходный материал обеспечиваться в соответств...,NaN,определяться при составление профессионально-о...,"1. puhta, h., thaine, g., doff, a., stranks, j...",наиболее полный перечень информационный источн...,"ибрагимов динар хамидовна, кафедра иностранный..."
5,санкт-петербургский государственный университе...,NaN,дисциплина «математический статистика» являтьс...,программа курс предназначить обучаться на 4 ку...,обучаться должны: • знать содержание дисциплин...,аудиторный учебный работа: теоретический занят...,NaN,NaN,NaN,тема для изучение и обсуждение 1) классификаци...,...,стандартно оборудовать лекционный аудитория. д...,стандартный аудиторный оборудование и по.,не требуется.,не требуется.,мел для доски.,NaN,1. боровков а.а. математический статистика. - ...,1. крамер г. математический метод статистики. ...,не требуется.,"никитин яков юрьевич, д.ф-м.н, проф. кафедра т..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,санкт-петербургский государственный университе...,NaN,дисциплина «русский язык как иностранный» пред...,уровень подготовленность обучаться в область р...,в результат освоение программа по траектория 1...,основной форма обучение русский язык являться ...,NaN,NaN,NaN,каждый учебный год иметь семестровый и модульн...,...,a) для проведение занятие по русский язык целе...,"наличие аудиовизуальный техники, включая мульт...",мультимедийный учебный комплекс с сильный прог...,1) компьютерный программы: microsoftoffice (в ...,возможность ксерокопирование учебный материалов.,"барыкин а.н., добровольский в.в.изучать глагол...",NaN,аникин м.н. си

In [30]:
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import pymorphy2
import my_parser
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
import re


obr_1 = 'рпд-мм/001292_Теория представлений.docx'
# obr_2 = 
# obr_3 =

cols = [column for column in mm]

def preproc_rpd(text):
    tokenizer = ToktokTokenizer()
    stopword_list = nltk.corpus.stopwords.words('russian')
    stop = set(stopwords.words('russian'))
    text = text['text']
    morph = pymorphy2.MorphAnalyzer()
    text = ' '.join([morph.parse(word)[0].normal_form for word in text.split()])
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    cleaned_tokens = [token for token in tokens if token.lower() not in stopword_list]
    cleaned_text = ' '.join(cleaned_tokens)           
    return cleaned_text

def process(obr):
    rpd_content = my_parser.feed_content(obr)
    rpd = pd.DataFrame(rpd_content)
    rpd = rpd.drop(index='title')
    rpd = rpd.apply(preproc_rpd)
    return rpd
    
def check(obr):
    obr = process(obr)
    closest = dict()
    mm = pd.read_csv('math-mech upd.csv')
    mm = mm.fillna('')
    
    for c in cols:
        closest[c] = list()
        min = 1000
        for i in range(1, len(mm)):
            v_1 = model.infer_vector(obr[c].split()).reshape(1, -1)
            v_2 = model.infer_vector(mm.loc[i][c].split()).reshape(1, -1)
            cos_dist = 1 - cosine_similarity(v_1, v_2)
            if cos_dist < min:
                min = cos_dist
                closest[c].append([i, cos_dist[0]])
    return closest


check(obr_1)

{'Титульная страница': [[1, array([0.37985086], dtype=float32)],
  [3, array([0.23240203], dtype=float32)],
  [7, array([0.1986165], dtype=float32)],
  [11, array([0.19774151], dtype=float32)],
  [13, array([0.1635499], dtype=float32)],
  [84, array([0.1552943], dtype=float32)],
  [140, array([0.1280362], dtype=float32)],
  [461, array([0.11902487], dtype=float32)],
  [753, array([0.11812091], dtype=float32)],
  [983, array([0.11545682], dtype=float32)]],
 'Раздел 1.': [[1, array([1.196864], dtype=float32)],
  [3, array([0.8104235], dtype=float32)],
  [10, array([0.7112871], dtype=float32)],
  [12, array([0.687915], dtype=float32)],
  [28, array([0.633578], dtype=float32)],
  [143, array([0.5636013], dtype=float32)],
  [447, array([0.4855095], dtype=float32)],
  [578, array([0.47573876], dtype=float32)],
  [670, array([0.4695542], dtype=float32)]],
 '1.1.': [[1, array([0.65061957], dtype=float32)],
  [4, array([0.6361917], dtype=float32)],
  [5, array([0.6088736], dtype=float32)],
  [7

In [55]:
obr_1 = 'рпд-мм/001292_Теория представлений.docx'
obr_2 = 'рпд-мм/002051 Практикум на ЭВМ.docx'
r_1 = process(obr_1)
r_2 = process(obr_1)
r_3 = process(obr_2)
for c in cols:
    if r_1[c] != '' and r_2[c] != '' and r_3[c] != '':
        v_1 = model.infer_vector(r_1[c].split()).reshape(1, -1)
        v_2 = model.infer_vector(r_2[c].split()).reshape(1, -1)
        v_3 = model.infer_vector(r_3[c].split()).reshape(1, -1)
#         cos_dist = cosine_similarity(v_1, v_2)
        print(c)
        print('Itself', cosine_similarity(v_1, v_1))
        print('Itself but diff doc', cosine_similarity(v_1, v_2))
        print('Different', cosine_similarity(v_1, v_3))
        print('Different 2', cosine_similarity(v_2, v_3))
        

Титульная страница
Itself [[1.]]
Itself but diff doc [[0.71561784]]
Different [[0.6422959]]
Different 2 [[0.7089316]]
1.1.
Itself [[1.]]
Itself but diff doc [[0.8692578]]
Different [[0.18414405]]
Different 2 [[0.1822294]]
1.2.
Itself [[0.9999999]]
Itself but diff doc [[0.85510606]]
Different [[-0.04125512]]
Different 2 [[-0.17423163]]
1.3.
Itself [[1.0000002]]
Itself but diff doc [[0.89856815]]
Different [[0.22485656]]
Different 2 [[0.37749404]]
1.4.
Itself [[0.9999999]]
Itself but diff doc [[0.82788765]]
Different [[0.1720317]]
Different 2 [[0.13900204]]
2.2.
Itself [[1.]]
Itself but diff doc [[0.9720668]]
Different [[0.46590048]]
Different 2 [[0.4807939]]
3.1.1
Itself [[1.]]
Itself but diff doc [[0.9495105]]
Different [[0.11863828]]
Different 2 [[0.15848842]]
3.1.2
Itself [[0.99999994]]
Itself but diff doc [[0.9184668]]
Different [[0.64386445]]
Different 2 [[0.6290479]]
3.1.3
Itself [[1.]]
Itself but diff doc [[0.9501041]]
Different [[0.45781106]]
Different 2 [[0.48533112]]
3.1.4
Its

# Дообучение на вспомогательных текстах

In [18]:
texts = pd.read_csv('posts.csv')

In [19]:
texts = texts.dropna()

In [20]:
texts = texts.to_numpy().reshape(3834)

In [22]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

vector_size = 30
documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(texts)]
model = Doc2Vec.load("pm_fmkn_trained upd")
# model.build_vocab(documents)

In [23]:
for epoch in range(10):
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=model.epochs)

In [24]:
model.save("pm_fmkn_trained upd 1")

# Проверка дообученной модели

In [25]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("pm_fmkn_trained upd 1")

In [31]:
obr_1 = 'рпд-мм/001292_Теория представлений.docx'
obr_2 = 'рпд-мм/002051 Практикум на ЭВМ.docx'
r_1 = process(obr_1)
r_2 = process(obr_1)
r_3 = process(obr_2)
for c in cols:
    if r_1[c] != '' and r_2[c] != '' and r_3[c] != '':
        v_1 = model.infer_vector(r_1[c].split()).reshape(1, -1)
        v_2 = model.infer_vector(r_2[c].split()).reshape(1, -1)
        v_3 = model.infer_vector(r_3[c].split()).reshape(1, -1)
#         cos_dist = cosine_similarity(v_1, v_2)
        print(c)
        print('Itself', cosine_similarity(v_1, v_1))
        print('Itself but diff doc', cosine_similarity(v_1, v_2))
        print('Different', cosine_similarity(v_1, v_3))
        print('Different 2', cosine_similarity(v_2, v_3))

Титульная страница
Itself [[1.]]
Itself but diff doc [[0.95758927]]
Different [[0.9294164]]
Different 2 [[0.92462015]]
1.1.
Itself [[0.99999994]]
Itself but diff doc [[0.9727106]]
Different [[0.5104247]]
Different 2 [[0.5506269]]
1.2.
Itself [[1.]]
Itself but diff doc [[0.9574197]]
Different [[0.7532395]]
Different 2 [[0.68672264]]
1.3.
Itself [[1.]]
Itself but diff doc [[0.97281736]]
Different [[0.44356826]]
Different 2 [[0.5003116]]
1.4.
Itself [[1.0000001]]
Itself but diff doc [[0.95789814]]
Different [[0.3144285]]
Different 2 [[0.34960264]]
2.2.
Itself [[1.]]
Itself but diff doc [[0.98820233]]
Different [[0.62388635]]
Different 2 [[0.6533715]]
3.1.1
Itself [[1.]]
Itself but diff doc [[0.84500647]]
Different [[0.2893311]]
Different 2 [[0.37586936]]
3.1.2
Itself [[1.]]
Itself but diff doc [[0.92576426]]
Different [[0.67999834]]
Different 2 [[0.75819147]]
3.1.3
Itself [[1.]]
Itself but diff doc [[0.9860833]]
Different [[0.6131046]]
Different 2 [[0.59108377]]
3.1.4
Itself [[1.]]
Itself